## Imports

In [1]:
# Imports and set torch device
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt
import kaolin as kal
import cv2
from collections import defaultdict
import trimesh
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F
from meshseg.mesh import MeshNormalizer


if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print('Torch will run on:', device)

object = 'bed' 
obj_path = 'data/demo/' + object + '.obj'

Torch will run on: cuda:0


## Read Mesh

In [2]:
# Read mesh
mesh = kal.io.obj.import_mesh(
    obj_path,
    with_normals=True,
    with_materials=False,
)


vertices_tensor = mesh.vertices.to(device)
faces_tensor = mesh.faces.to(device)

vertices = vertices_tensor.detach().cpu().numpy()
faces = faces_tensor.detach().cpu().numpy()
colors =  mesh.vertex_normals.cpu().numpy()

print('Number of vertices: ', vertices.shape[0])
print('Number of faces: ', faces.shape[0])

Number of vertices:  11973
Number of faces:  19568


## Normalize Mesh

In [3]:
shift = torch.mean(vertices_tensor, dim=0)
scale = torch.max(torch.norm(vertices_tensor-shift, p=2, dim=1))
vertices_tensor = (vertices_tensor-shift) / scale
vertices = vertices_tensor.detach().cpu().numpy()

## Import Point Cloud

In [4]:
point_cloud = np.load(f'outputs/demo/ABO/{object}/point_cloud.npy')
p = mp.plot(point_cloud, shading={'point_size':0.1, 'point_color':'black'})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.002281…

## Import BB Casting

In [5]:
view=8
prompt='headboard'
pt_size=0.12
included_pts = np.load(f'outputs/demo/ABO/{object}/included_pts/view_{view}_the {prompt} of a {object}.npy')
p = mp.plot(point_cloud, shading={'point_size':pt_size, 'point_color':'black'}, return_plot=True)
p.add_points(point_cloud[included_pts].reshape(-1,3), shading={'point_size':pt_size, 'point_color':'blue'})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.002281…

In [6]:
import json
colors_dict = {
    0: [1, 0, 0],   # Red
    1: [0, 1, 0],   # Green
    2: [0, 0, 1],   # Blue
    3: [1, 1, 0],   # Yellow
    4: [1, 0, 1],   # Magenta
    5: [0, 1, 1],   # Cyan
    6: [0.5, 0, 0], # Dark Red
    7: [0, 0.5, 0], # Dark Green
    8: [0, 0, 0.5], # Dark Blue
    9: [0.5, 0.5, 0.5] # Gray
}
colors_lst = ['Red', 'Green', 'Blue', 'Yellow', 'Magenta', 'Cyan', 'Dark Red', 'Dark Green', 'Dark Blue', 'Grey']
output_file_per_sample = open(f'./outputs/demo/ABO/{object}/samples_preds.json')
output_file_per_face = open(f'./outputs/demo/ABO/{object}/face_preds.json')
output_per_sample = np.array(json.load(output_file_per_sample))
output_per_face = np.array(json.load(output_file_per_face))
segments = np.unique(output_per_sample)
segments_idx_per_sample = np.unique(output_per_sample, return_inverse=True)[1]
segments_idx_per_face = np.unique(output_per_face, return_inverse=True)[1]
prompts = segments.tolist()
if ('unknown' in prompts):
    prompts.remove('unknown')
# print('Prompts: ', prompts)
colors_per_sample = np.array([colors_dict[segment_id] for segment_id in segments_idx_per_sample])
colors_per_face = np.array([colors_dict[segment_id] for segment_id in segments_idx_per_face])
for i in range(len(segments)):
    print(colors_lst[i],':\t',segments[i])
mp.plot(point_cloud, c=colors_per_sample, shading={'point_size':pt_size})
plt.show()

Red :	 bed frame
Green :	 headboard
Blue :	 legs
Yellow :	 mattress
Magenta :	 unknown


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.002281…

In [7]:
p = mp.plot(vertices, faces, c = colors_per_face, return_plot=True)
p.add_points(point_cloud, c = colors_per_sample, shading={'point_size':0.06})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.002162…

1